In [1]:
import pandas as pd
import numpy as np
import pyreadr
import seaborn as sns
from sklearn.preprocessing import scale

### 1. Load the state legislative professionalism data from the folder. 

In [2]:
#convert R file into a pandas df 
result = pyreadr.read_r('legprof-components.v1.0.RData') 
df = result['x']

In [3]:
df

,fips,stateabv,state,sessid,t_slength,slength,salary_real,expend,year,mds1,mds2
0,1,AL,Alabama,1973/4,46.000000,36.000000,1.768022,125.097298,1974.0,-1.706181,0.384820
1,1,AL,Alabama,1975/6,110.000000,74.000000,2.933038,203.846588,1976.0,-1.212882,-0.081507
2,1,AL,Alabama,1977/8,83.000000,60.000000,2.082810,184.011520,1978.0,-1.414966,0.127900
3,1,AL,Alabama,1979/80,65.000000,60.000000,1.694951,175.986252,1980.0,-1.543154,0.272688
4,1,AL,Alabama,1981/2,218.680008,149.100006,3.472914,204.123642,1982.0,-0.501364,-1.003878
5,1,AL,Alabama,1983/4,188.860008,149.100006,11.705946,206.674492,1984.0,-0.584019,-0.741324
6,1,AL,Alabama,1985/6,206.400009,149.100006,11.210624,324.724503,1986.0,-0.382062,-0.776330
7,1,AL,Alabama,1987/7,121.550003,104.550003,1.983966,289.746048,1988.0,-1.081545,-0.097162
8,1,AL,Alabama,1989/90,109.550003,103.550003,1.753740,284.295959,1990.0,-1.166710,-0.000710
9,1,AL,Alabama,1991/2,126.550003,104.550003,1.659808,315.513412,1992.0,-1.032720,-0.114932


### 2. Munge the data:

a. select only the continuous features that should capture a state legislature’s level
of “professionalism” (session length (total and regular), salary, and
expenditures)

In [4]:
df.columns

Index(['fips', 'stateabv', 'state', 'sessid', 't_slength', 'slength',
       'salary_real', 'expend', 'year', 'mds1', 'mds2'],
      dtype='object')

In [5]:
df = df[["t_slength","slength",'salary_real', 'expend',"sessid","state"]]

b. restrict the data to only include the 2009/10 legislative session for consistency;


In [6]:
year_2009_10_filter = df["sessid"].isin(["2009/10"])
df_filtered = df[year_2009_10_filter]

In [7]:
df_filtered

,t_slength,slength,salary_real,expend,sessid,state
18,116.550003,104.550003,1.050421,535.142319,2009/10,Alabama
37,128.510002,127.800003,74.805863,1493.835083,2009/10,Alaska
56,286.129990,197.379997,48.393666,631.132935,2009/10,Arizona
75,80.230000,80.230000,30.669025,516.637619,2009/10,Arkansas
94,390.000000,270.000000,213.405133,5523.100830,2009/10,California
113,205.199997,205.199997,60.492082,440.449768,2009/10,Colorado
132,160.910004,144.260002,56.459277,577.087280,2009/10,Connecticut
151,93.709999,93.000000,85.131219,277.078888,2009/10,Delaware
170,101.529995,90.169998,60.530594,2087.293091,2009/10,Florida
189,80.000000,80.000000,34.968457,241.597389,2009/10,Georgia


c. omit all missing values;

In [8]:
df_filtered = df_filtered.dropna()
df_filtered = df_filtered.reset_index()
df_filtered = df_filtered.drop(columns=["sessid"])


d. standardize the input features;

In [9]:
df_filtered_state = df_filtered[["state"]]
df_filtered_scale = df_filtered[['t_slength', 'slength', 'salary_real', 'expend']]
columns = [['t_slength', 'slength', 'salary_real', 'expend',"state"]]
final_df = pd.DataFrame(scale(df_filtered_scale), index=df_filtered_scale.index, columns=df_filtered_scale.columns)
final_df = np.hstack([final_df,df_filtered_state])
final_df = pd.DataFrame(final_df, columns=columns)

In [10]:
final_df

,t_slength,slength,salary_real,expend,state
0,-0.375511,-0.464234,-1.10332,-0.242478,Alabama
1,-0.231786,-0.146736,0.40529,0.868023,Alaska
2,1.66236,0.803436,-0.13495,-0.131287,Arizona
3,-0.811974,-0.796343,-0.497493,-0.263913,Arkansas
4,2.91058,1.79512,3.24023,5.53532,California
5,0.689809,0.910225,0.112513,-0.352165,Colorado
6,0.157569,0.0780389,0.0300258,-0.193891,Connecticut
7,-0.649983,-0.621959,0.616487,-0.541406,Delaware
8,-0.556009,-0.660605,0.113301,1.55545,Florida
9,-0.814738,-0.799484,-0.409551,-0.582506,Georgia


e. and anything else you think necessary to get this subset of data into workable
form (hint: consider storing the state names as a separate object to be used in plotting later)

Kept state name column to use for plotting 